In [ ]:
import numpy as np
import pandas as pd
import torch

pd.options.display.max_colwidth = 200

In [ ]:
file = '/root/logs/results.csv'
df = pd.read_csv(file, header=0, index_col=0)
# display(df.head())b2
# print(df.shape)
df.drop(['gpu-id','queue-id','queue-index','tid','grd','wgr','scr','vgpr','sgpr','fbar','sig'], axis=1, inplace=True)
# df.dropna(subset=['registersPerThread','staticSMem','dynamicSMem','size','throughput'], how='all', inplace=True)

# demangling the name
# df['KernelName'] = df['KernelName'].apply(torch._C._demangle)

df['DurationUs'] = (df['EndNs'] - df['BeginNs']) / 1000.0
print(df.shape)
display(df.head(25))

In [ ]:
# df['KernelName'].to_csv('/root/DeepLearningMisc/microbench/kernelnames-results.csv', index = False)

In [ ]:
print(df.dtypes)
# print(df.tail(50))

In [ ]:
df = df[~df['KernelName'].str.contains("hip_fill_n")]
df.reset_index(inplace=True, drop=True)
print(df.shape)

In [ ]:
# print(df[(df['size'] > 2.0) & (df['size'] < 2.5) & (df['name'] == '[CUDA memcpy HtoD]')])
accComputeIndices = df.index[df['KernelName'].str.contains("MaxValuePair")].tolist()
display(accComputeIndices)

In [ ]:
t1 = df.loc[:accComputeIndices[0]-1] 
t2 = df.loc[accComputeIndices[0]+1:accComputeIndices[1]-1]
t3 = df.loc[accComputeIndices[1]+1:accComputeIndices[2]-1]
t4 = df.loc[accComputeIndices[2]+1:accComputeIndices[3]-1]
t5 = df.loc[accComputeIndices[3]+1:accComputeIndices[4]-1]
t6 = df.loc[accComputeIndices[4]+1:accComputeIndices[5]-1]
t7 = df.loc[accComputeIndices[5]+1:accComputeIndices[6]-1]
print(t1.shape, t2.shape, t3.shape, t4.shape, t5.shape, t6.shape, t7.shape)

print((t1['EndNs'].max() - t1['BeginNs'].min())/1e6,
      (t2['EndNs'].max() - t2['BeginNs'].min())/1e6,
      (t3['EndNs'].max() - t3['BeginNs'].min())/1e6,
      (t4['EndNs'].max() - t4['BeginNs'].min())/1e6,
      (t5['EndNs'].max() - t5['BeginNs'].min())/1e6,
      (t6['EndNs'].max() - t6['BeginNs'].min())/1e6,
      (t7['EndNs'].max() - t7['BeginNs'].min())/1e6)

In [ ]:
# start would be 0 or accCompute+4
start = accComputeIndices[3] + 1
end = accComputeIndices[4] - 1
b2df = df.loc[start:end,:]
b2df.reset_index(inplace=True, drop=True)

# b2df = b2df[~((b2df['name'] == '[CUDA memset]') | (b2df['name'] == '[CUDA memcpy DtoD]') | (b2df['name'] =='[CUDA memcpy DtoH]'))]
print(b2df.shape)
display(b2df.head())
display(b2df.tail())

In [ ]:
# input df should be a dataframe containing entries corresponding to a batch
def isNativeRow(name):
    """
    Returns True if a pytorch native operation is performed based on string matching with name attribute.
    """
    return ("at::native::") in name and ("fill_kernel_cuda" not in name)

def isFixIndexJoinID(index):
    """Return True if index in list. We want to increment joinID by 1 for these to sync with pytorch trace"""
#     return False # disable
#     ## Resnet__
#     indices = [7717,7759,7824,8220, # convolutions immediately following batchnorm'''
#                8264,8265, # softmax fwd, loss
#                8582,8583,8584] # loss bwd, softmax bwd, conv bwd

#     ## Resnext101_32x8d forward
#     indices = [18, 70, 141, 537, # convolutions immediately following batchnorm
#                581] # softmax fwd
#     ## Resnext101_32x8d fp16 forward
#     indices = [16, 121, 243, 699, #convolutions immediately following batchnorm
#                743] # softmax fwd

#     ## Resnext101_32x4d forward
#     indices = [16, 62, 127, 523, # convolutions immediately following batchnorm
#                567] # softmax fwd
#     return index in indices
#     ## Resnext101_32x4 fp16, Resnext101_64x4d fp16
#     # mi50
#     indices =[16, 121, 241, 609, 652, 653, 654, 656, 657, 658] # substract 652
#     # mi100
#     indices = [16, 121, 243, 699, 742, 743, 744, 746, 747, 748] # substract 742
#     return index in indices

#     ## Resnext101_64x4d forward
#     indices = [18, 70, 204, 695, # convolutions immediately following batchnorm
#                739] # softmax fwd
#     return index in indices

#     ## Resnet50v1.5 fwd+bwd
#     indices = [19, 61, 117, 204, 248, 249, 268]               
#     return index in indices


def addJoinIDs(df):
    df['joinID'] = np.nan
    joinID_loc = df.columns.get_loc('joinID')
    joinid = 0
    
    isCurNative = False
    wasPrevNative = False
    
    for index, row in df.iterrows():
        # do something
        if ("hip_fill_n" in row['KernelName']) or ("fill_kernel_cuda" in row['KernelName']):
            continue
            
        isCurNative = isNativeRow(row['KernelName'])
        if isCurNative or wasPrevNative or isFixIndexJoinID(index) :
#             if index != 742:
#                 joinid +=1
            joinid +=1
        
        df.loc[index, 'joinID'] = joinid
        
        wasPrevNative = isCurNative and ("copy_kernel_impl" not in row['KernelName'])

        
#     df['joinID'] = df['joinID'].astype(int)

addJoinIDs(b2df)
display(b2df)

In [ ]:
b2df.to_csv('/root/logs/mi100/resnext101_32x4d-fp16-amd-fix.csv')

In [ ]:
till = b2df.index[b2df['KernelName'].str.contains("softmax_warp_forward")].tolist()
assert len(till)==1, "Error"
fwd = b2df.loc[:till[0]]
# display(fwd.loc[:50, ['KernelName', 'joinID']])
display(fwd)
# fwd.to_csv('/root/logs/resnext101_32x8d-fwd.csv')

In [ ]:
# b2df = fwd # hack for aggregating just for fwd pass
b2dfJoin = b2df.dropna(subset=['joinID'])
b2dfJoin['joinID'] = b2dfJoin['joinID'].astype(int)
print(b2dfJoin.shape)

def concatKernelNames(x):
    return "[%s]" % ', '.join(x)
concatKernelNames.__name__ = 'Kernels'

b2agg = b2dfJoin.groupby('joinID').agg({'KernelName': [concatKernelNames, 'count'],
                                    'BeginNs': 'min',
                                    'DurationUs': 'sum'})
# 'EndNs': 'max'
b2agg.columns = pd.Index(['kernelNames','numKernels','minBeginNs','kernelDuration'])
b2agg['duration'] = (b2agg['minBeginNs'].shift(-1, axis=0) - b2agg['minBeginNs']) / 1000.0
# b2agg['duration'] = (b2agg['maxEndNs'] - b2agg['minBeginNs']) / 1000.0

# display(b2agg[b2agg['numKernels']>1])
# print(len(b2agg))
display(b2agg)

In [ ]:
def kernelWithTime(group):
    infoKernels = []
    for kernel, end, start in zip(group['KernelName'], group['EndNs'], group['BeginNs']):
        info = kernel + "[" + str((end - start)/1000) + "]"
        infoKernels.append(info)
    return infoKernels

b2agg['kernelNames'] = b2dfJoin.groupby('joinID').apply(kernelWithTime)

In [ ]:
display(b2agg)

In [ ]:
b2agg.to_csv('/root/logs/resnext101_32x4d-fp16-batch-amd-1386s877m.csv')